### 1. Non-dominated Sorting Genetic Algorithm (NSGA-II)
##### 2. Multi-Objective Particle Swarm Optimization (MOPSO)
##### 3. Multi-Objective Differential Evolution (MODE)
##### 4. Multi-Objective Genetic Programming (MOGP)

In [2]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from networks import get_all_devices_combined
from services import get_all_services
import pandas as pd

# Define the problem as a multi-objective optimization problem
creator.create("FitnessMulti", base.Fitness, weights=[-1.0, -1.0])
creator.create("Individual", list, fitness=creator.FitnessMulti)

physical_machines = get_all_devices_combined("config-1")
services = get_all_services("config-1")

# Define the problem-specific parameters
n = len(physical_machines)  # Number of physical machines
s = len(services)  # Number of services

# Values for num_generations and alpha
num_generations_values = [10, 20, 50, 100, 200, 500, 1000]
alpha_values = [i / 10.0 for i in range(1, 10)]  # 0.1 to 0.9 with 0.1 increments

# Create an empty DataFrame to store the results
columns = ['generations', 'alpha', 'latency', 'energy', 'placement_physical']
results_df = pd.DataFrame(columns=columns)

# Outer loop: iterate through num_generations values
for num_generations in num_generations_values:
    # Inner loop: iterate through alpha values
    for alpha in alpha_values:
        
        beta = 1 - alpha   # Weight for energy consumption
        
        # Define parameters. This will be loaded from the database.
        # Here, we use random values for demonstration purposes.
        # Replace these with actual data when implementing in a real system.
        
        # Variable number of virtual machines for each physical machine
        max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))
        
        ################ Physical and virtual machine configurations ##################
        
        # Initialization
        R = np.empty(shape=(n, max_v))  # Request network delay of the machine
        R.fill(999999) # Initialize with very large value
        S = np.empty(shape=(n, max_v))  # Response network delay of the machine
        S.fill(999999) # Initialize with very large value
        I = np.empty(shape=(n, max_v))  # IPS currently executed
        I.fill(0) # Initialize with 0
        X = np.empty(shape=(n, max_v))  # Maximum IPS possible
        X.fill(1) # Initialize with 1
        
        PI = np.empty(shape=(n, max_v))  # Power consumption in idle
        PI.fill(999) # Initialize with very large value
        PM = np.empty(shape=(n, max_v))  # Maximum power consumption
        PM.fill(999) # Initialize with very large value
        
        # Fill in with the actual numbers
        for i in range(len(physical_machines)):
            for j in range(len(physical_machines[i].guest_machines)):
                R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
                S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
                X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
                PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
                PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization
        
        ################ Service configurations ###################
        
        # Acceptable latencies for services
        SAL = [service.acceptable_latency for service in services]
        # Initialize IPS values for services (replace with actual data)
        SIPS = [service.average_instructions_per_second for service in services]
        # Layers considered for service
        SL = [service.layer for service in services]
        
        # Define the objective functions
        def evaluate_individual(individual):
            # Calculate energy consumption based on the parameters
            E = PI + (PM - PI) * (I / X)
            total_latency = 0.0
            total_energy = 0.0
            success_count = 0
            for i in range(s):
                p = individual[i]  # Physical machine where service is placed
                if p < n:
                    for v in range(len(physical_machines[p].guest_machines)):
                        vm = physical_machines[p].guest_machines[v]
                        latency = R[p][v] + (SIPS[i] / X[p][v]) + S[p][v]
                        if SL[i] is not None and SL[i] != vm.layer: # Consider the layer constrains provided by the service
                            continue
                        if latency <= SAL[i]:
                            total_latency += latency
                            current_instructions = PI[p][v] + SIPS[i]
                            E[p][v] = PI[p][v] + (PM[p][v] - current_instructions) * (I[p][v] / X[p][v]) # Update current energy consumption
                            total_energy += E[p][v]  # Calculate energy consumption
                            success_count += 1
            
            # if all the services are not placed for this particular individual,
            # return a large fitness value to ignore it
            if success_count is not len(services):
                total_latency = 1000
                total_energy = 1000
                
            # Calculate the weighted sum of objectives
            weighted_latency = alpha * total_latency
            weighted_energy = beta * total_energy
            return np.array([weighted_latency, weighted_energy])
        
        # Create a DEAP toolbox and register functions
        toolbox = base.Toolbox()
        toolbox.register("attr_int", random.randint, 0, n - 1)
        toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_int,), n=s)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        
        # Evaluation function
        toolbox.register("evaluate", evaluate_individual)
        toolbox.register("mate", tools.cxTwoPoint)
        toolbox.register("mutate", tools.mutUniformInt, low=0, up=n - 1, indpb=0.2)
        toolbox.register("select", tools.selBest)
        
        # Crossover, mutation, and selection registration

        # Create the initial population
        population = toolbox.population(n=100)
    
        # Create statistics object to track performance
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean, axis=0)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)
    
        # Create a logbook to log statistics
        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "avg", "min", "max"
    
        # Run NSGA-II algorithm
        algorithms.eaMuPlusLambda(population, toolbox,mu=100, lambda_=200, cxpb=0.7, mutpb=0.2, ngen=num_generations, stats=stats,
                            halloffame=None, verbose=True)
    
        # Extract the Pareto front solutions (not necessarily Pareto-optimal in a basic GA)
        pareto_front = tools.sortNondominated(population, len(population), first_front_only=True)[0]
        
        # Get the fitness values and individual of the last generation
        ind = pareto_front[len(pareto_front) - 1]

        # Append the data to the results DataFrame
        results_df = results_df._append({'generations': num_generations,
                                        'alpha': alpha,
                                        'latency': ind.fitness.values[0],
                                        'energy': ind.fitness.values[1],
                                        'placement_physical': ind}, ignore_index=True)
            
# Save the results DataFrame to a CSV file
results_df.to_csv('results.csv', index=False)

/Library/Python/3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Library/Python/3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/var/folders/0c/45yxrbkn6sx_nd6pdck7bc940000gn/T/ipykernel_87379/3301961904.py:147: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({'generations'

gen	nevals	avg        	min        	max        
0  	100   	[100. 900.]	[100. 900.]	[100. 900.]
1  	183   	[ 99.29936222 897.72473446]	[ 29.9362219 672.473446 ]	[100. 900.]
2  	181   	[ 99.29936222 897.72473446]	[ 29.9362219 672.473446 ]	[100. 900.]
3  	174   	[ 99.29936222 897.72473446]	[ 29.9362219 672.473446 ]	[100. 900.]
4  	182   	[ 99.29936222 897.72473446]	[ 29.9362219 672.473446 ]	[100. 900.]
5  	182   	[ 99.29936222 897.72473446]	[ 29.9362219 672.473446 ]	[100. 900.]
6  	181   	[ 98.59872444 895.44946892]	[ 29.9362219 672.473446 ]	[100. 900.]
7  	180   	[ 98.59872444 895.44946892]	[ 29.9362219 672.473446 ]	[100. 900.]
8  	184   	[ 97.20447399 890.47687189]	[ 29.9362219  630.26685076]	[100. 900.]
9  	179   	[ 95.03709207 882.64511306]	[ 23.38936444 571.87722549]	[100. 900.]
10 	181   	[ 91.51323671 870.91510745]	[ 23.38936444 571.87722549]	[100. 900.]
gen	nevals	avg        	min        	max        
0  	100   	[200. 800.]	[200. 800.]	[200. 800.]
1  	176   	[200. 800.]	[200. 800.]	[